# Report of Project 05: Implementation and Evaluation of Machine Learning (Support Vector Machine) Segmentation
## Data analysis project - B.Sc. Molecular Biotechnology Heidelberg University
### 19.07.21
### Authors: Michelle Emmert, Juan Hamdan, Laura Sanchis and Gloria Timm

*ich werde kommentare, anmerkungen, noch zu klärende fragen... immer in kursiv schreiben*

*Idee: unseren algorithmus anhand eines bildes erklären und dieses immer wieder im report zeigen um die veränderungen
zu zeigen & am ende dice berechnen --> veranschaulichen*

*Unterüberschriften noch einfügen*

# Abstract





# Table of contents *ggf. noch 2.1, 2.2 etc.*
**1. Introduction** <br>
**2. Our Datasets** <br>
**3.** <br>
**4. Pre-processing** <br>
**5. Implementation of support vector machine** <br>
**6. Evaluation using the Dice coefficient** <br>
**7. Results** <br>
**8. Discussion** <br>
**9. Bibliography**





# 6. Dice Coefficient
## 6.1 The Theory Behind the Dice Coefficient
The Dice coefficient is a score to evaluate and compare the accuracy of a segmentation.
Needed for its calculation are the segmented image, as well as a corresponding binary reference point also called
ground truth (Bertels et al., 2019).
As a ground truth image, researchers mostly use the segmentation result of humans. We will use the ground truth images
provided with our data sets, which we suspect to be acquired by this method.
Using the ground truth image, the labels true positive (TP), false positive (FP) and false
negative (FN) are assigned to each pixel of the segmented image (Menze et al., 2015).
This information is then used to calculate the dice coefficient using formula (1):

(1) dice = ${\frac{2TP}{2TP + FP + FN}}$ \varepsilon [0,1] <br>
(Menze et al., 2015)

A dice score of 0 indicates that the ground truth and the segmentation result do not have any overlap. A dice score of 1 on the
other hand, shows a 100% overlap of ground truth and segmented image (Bertels et al., 2019).

## 6.2 Implementing the dice coefficient

In [1]:
import Finalmodules.dicescore as dicescore


ModuleNotFoundError: No module named 'Finalmodules.dicescore'

## 6.3 Synthetic Images
### 6.3.1 Definition and Goal
The concept behind creating synthetic images, is to use algorithms and images, which are already available to generate
new ones. (Dunn et al., 2019) Although our first objective was to just use these new images to test our code for the dice score, we realized
while researching for this topic that synthetic images have an immense potential, most of all for the training of machine
learning algorithms. By expanding our training set with diverse images of good quality, we expect a more accurate model (Mayer et al., 2017 ALTERNATIV: https://arxiv.org/abs/1807.07428).
In order to train our model and test it afterwards with new data, we have to split up our dataset of 28 cell images
between those two sets, which leads to a training set of less than 28 images. Because of this, we decided to implement
our synthetically produced images not only to test the dice score, but to enlarge our training data pool for our
Support Vector Machine, and afterwards, check if its efficiency was improved with our dice score. There are many methods
that can be used in order to generate synthetic images (Ward et al., 2019). Because of the scope of our project and the
kind of images we want to produce, we focused on image composition and domain transfer.

### 6.3.2 Image Composition
Image composition consists of taking various foreground images, which have been segmented out of their backgrounds or
have a .png format to begin with, and paste them onto different backgrounds (Tripathi, 2019). The foreground images can
be modified using different light conditions, contrasts, zooms or rotations in order to achieve more variety in the results (???).
--> probably not as useful for our case as cells are usually in front of dark background, but still an option to evaluate
--> will the Dice Score get better with that method?

*Here we could insert the code from my Jupyter notebook -> issue: it hasn't been written by us
(and it hasn't been modified either...), so maybe it would be better to just use our own code when we write it.*

### 6.3.3 Domain randomization
In domain randomization there is a model from the object class you want to train your model for, and in that model,
every parameter from the object and its environment that is not necessary for its recognition by the machine has been
randomized. This means for example size, lighting or color, and there are very powerful tools to do this, like Unity or
Blender. <br>
*--> probably more useful, as we have changes in size, dividing or leaving cells etc. However, it is usually
used to train robots to work from a simulation to reality, so it might be a bit too much.*

# Support Vector Machine

1. Tensorflow --> an Option?


**9. Bibliography**
Menze BH, Jakab A, Bauer S, Kalpathy-Cramer J, Farahani K, Kirby J, Burren Y, Porz N, Slotboom J, Wiest R, Lanczi L,
Gerstner E, Weber MA, Arbel T, Avants BB, Ayache N, Buendia P, Collins DL, Cordier N, Corso JJ, Criminisi A, Das T,
Delingette H, Demiralp Ç, Durst CR, Dojat M, Doyle S, Festa J, Forbes F, Geremia E, Glocker B, Golland P, Guo X, Hamamci A,
Iftekharuddin KM, Jena R, John NM, Konukoglu E, Lashkari D, Mariz JA, Meier R, Pereira S, Precup D, Price SJ, Raviv TR,
Reza SM, Ryan M, Sarikaya D, Schwartz L, Shin HC, Shotton J, Silva CA, Sousa N, Subbanna NK, Szekely G, Taylor TJ,
Thomas OM, Tustison NJ, Unal G, Vasseur F, Wintermark M, Ye DH, Zhao L, Zhao B, Zikic D, Prastawa M, Reyes M, Van Leemput K.
The Multimodal Brain Tumor Image Segmentation Benchmark (BRATS). IEEE Trans Med Imaging. 2015


Bertels, J., Eelbode, T., Berman, M., Vandermeulen D., Maes F., Bisschops, R., Blaschko, M. 2019.
Optimization for Medical Image Segmentation: Theory and Practice when evaluating with Dice Score or Jaccard Index


Dunn, K.W., Fu, C., Ho, D.J., Lee S., Han S., Salama P., Delp E. DeepSynth: Three-dimensional nuclear segmentation of
biological images using neural networks trained with synthetic data. Sci Rep 9, 18295 (2019)


Mayer, N., Ilg, E., Fischer, P., Hazirbas, C., Cremers, D., Dosovitskiy, A.,Brox, T.  What Makes Good Synthetic Training Data for Learning Disparity and Optical Flow Estimation?.
Int J Comput Vis 126, 942–960 (2018).


Ward, D.,Moghadam P.,Hudson, N. Deep Leaf Segmentation Using Synthetic Data
(2019) *welches journal?*

Tripathi, S., Chandra S., Agrawal, A., Tyagi, A., Rehg, J. M., Chari, V. (2019) Learning to Generate Synthetic
Data via Compositing *welches journal?*